# Film Critic Reviews Sentiment Analysis using LSTM

This project implements a sentiment analysis model using Long Short-Term Memory (LSTM) neural networks to classify movie reviews as positive or negative. The model is trained on the IMDB dataset of 50,000 movie reviews.

## Project Overview
- **Dataset**: IMDB Movie Reviews Dataset (50,000 reviews)
- **Model**: LSTM (Long Short-Term Memory) Neural Network
- **Framework**: TensorFlow/Keras
- **Accuracy**: ~87.89% on test data


In [1]:
!pip install kaggle


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importing the Dependencies

In [2]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data Collection - Kaggle API

In [5]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [6]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [7]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [8]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other




  0%|          | 0.00/25.7M [00:00<?, ?B/s]
100%|██████████| 25.7M/25.7M [00:00<00:00, 1.01GB/s]


In [9]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()

In [12]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


# Loading the Dataset

In [14]:
data = pd.read_csv("IMDB Dataset.csv")

In [15]:
data.shape

(50000, 2)

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [18]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [19]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_10736\2568826810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


# Data Preprocessing
From the replace command, we have changed positive to 1 and negative to 0

In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [21]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [22]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [23]:
# split the data into training data and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [24]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


# Text Tokenization and Padding

In [25]:
# Tokenzie text, data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)



In [26]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [27]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [28]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [29]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [30]:
print(Y_test)

33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


# LSTM Model - Long Short-Term Memory


In [31]:
# build the model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
# compiler the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Training the Model

In [34]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 122s 240ms/step - accuracy: 0.7294 - loss: 0.5325 - val_accuracy: 0.8413 - val_loss: 0.3637
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 123s 247ms/step - accuracy: 0.8450 - loss: 0.3689 - val_accuracy: 0.8305 - val_loss: 0.3812
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 173s 346ms/step - accuracy: 0.8741 - loss: 0.3139 - val_accuracy: 0.8460 - val_loss: 0.3587
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 189s 377ms/step - accuracy: 0.8945 - loss: 0.2703 - val_accuracy: 0.8449 - val_loss: 0.3691
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 191s 382ms/step - accuracy: 0.8955 - loss: 0.2654 - val_accuracy: 0.8625 - val_loss: 0.3318


# Model Evaluation

In [35]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 72ms/step - accuracy: 0.8663 - loss: 0.3237
Test Loss: 0.3198646903038025
Test Accuracy: 0.8669000267982483


# Building a Predictive System

In [36]:
def predict_sentiment(review):
  #tokenizer and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment="positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [37]:
# example usage
new_review = "This movie was fantastic. I loved it! "
sentiment = predict_sentiment(new_review)
print(f"The Sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
The Sentiment of the review is: positive


In [38]:
# example usage
new_review = "The film's stunning visuals and compelling storyline make it a must-watch. However, some might find the pacing a bit slow in the middle. "
sentiment = predict_sentiment(new_review)
print(f"The Sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
The Sentiment of the review is: positive


In [39]:
# example usage
new_review = "This movie ok but not that good. "
sentiment = predict_sentiment(new_review)
print(f"The Sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
The Sentiment of the review is: negative


In [40]:
# example usage
new_review = "The movie's plot was convoluted and hard to follow, leaving me disappointed. The acting felt forced and unconvincing throughout."
sentiment = predict_sentiment(new_review)
print(f"The Sentiment of the review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
The Sentiment of the review is: negative
